# 03 | Plot Fan Curve

In [1]:
from opencefadb import OpenCeFaDB

In [2]:
WORKING_DIR = "local-db"

In [3]:
# if you need help creating GRAPHDB:
# from h5rdmtoolbox.catalog import GraphDB

# graphdb = GraphDB(
#     endpoint="http://localhost:7200",
#     repository="OpenCeFaDB-Sandbox",
#     username="admin",
#     password="admin"
# )
# graphdb.get_or_create_repository("graphdb-config-sandbox.ttl")

In [ ]:
USE_GRAPH_DB = False

if USE_GRAPH_DB:
    db = OpenCeFaDB.from_graphdb_setup(
        working_directory=WORKING_DIR,
        version="latest",
        sandbox=True,
        endpoint="http://localhost:7200",
        repository="OpenCeFaDB-Sandbox",
        username="admin",
        password="admin",
        add_wikidata_store=False
    )
else:
    db = OpenCeFaDB.from_rdflib_setup(
        working_directory=WORKING_DIR,
        version="latest",
        sandbox=True
    )

# the first time we need also to download all metadata, upload the data and add the HDF infile index for faster queries:
for ttl_filename in db.rdf_directory.rglob("*.ttl"):
    db.rdf_store.upload_file(ttl_filename)
db.add_hdf_infile_index()

 [OpenCEFA DB]: Searching for the latest version...


You may download the files manually if the download fails:

In [ ]:
base = "https://doi.org/10.5281/zenodo.17572275#standard_name_table/"

sn_mean_nrot = base + "derived_standard_name/arithmetic_mean_of_fan_rotational_speed"
sn_mean_dp_stat = base + "derived_standard_name/arithmetic_mean_of_difference_of_static_pressure_between_fan_outlet_and_fan_inlet"
sn_mean_dp_tot = base + "derived_standard_name/arithmetic_mean_of_difference_of_total_pressure_between_fan_outlet_and_fan_inlet"
sn_mean_vfr = base + "derived_standard_name/arithmetic_mean_of_fan_volume_flow_rate"

operating_point_standard_names = {
    sn_mean_dp_stat,
    sn_mean_dp_tot,
    sn_mean_vfr
}

In [ ]:
rotation_speeds = [600, 800, 1000, 1200, 1400]
observation_clusters = []
for rs in rotation_speeds:
    observations = db.get_operating_point_observations(
        n_rot_speed_rpm=rs,
        operating_point_standard_names=operating_point_standard_names,
        standard_name_of_rotational_speed=sn_mean_nrot,
        n_rot_tolerance=0.05 # 5 %
    )
    observation_clusters.append(observations)

In [ ]:
from opencefadb.models.fan_curve import SemanticFanCurve

from opencefadb import plotting
import matplotlib.pyplot as plt

In [ ]:
fan_curves = [
    SemanticFanCurve.from_observations(
        observations=observations
    ) for observations in observation_clusters
]

In [ ]:
with plotting.SingleAxis(
        scale=1.0,
        filename="test_fan_curve.svg",
) as dax:
    for fc in fan_curves:
        fc.errorbar(
            x="arithmetic_mean_of_fan_volume_flow_rate",
            # y="arithmetic_mean_of_difference_of_static_pressure_between_fan_outlet_and_fan_inlet",
            y="arithmetic_mean_of_difference_of_total_pressure_between_fan_outlet_and_fan_inlet",
            xlabel=None,
            ylabel=None,
            label="$p_{tot} (n=600 rpm)$",
            marker=".",
            linestyle='-',
            ax=dax.ax,
            raise_on_no_data_points=False,
            verbose=True
        )
    # plt.legend()
    plt.tight_layout()

In [ ]:
n_rot = 1000
observations_1000 = db.get_operating_point_observations(
    n_rot_speed_rpm=n_rot,
    operating_point_standard_names=operating_point_standard_names,
    standard_name_of_rotational_speed=sn_mean_nrot,
    n_rot_tolerance=0.05 # 5 %
)

In [ ]:
fan_curve_1000 = SemanticFanCurve.from_observations(
    observations=observations_1000
)

In [ ]:
with plotting.SingleAxis(
        scale=1.0,
        filename="test_fan_curve.svg",
) as dax:
    fan_curve.errorbar(
        x="arithmetic_mean_of_fan_volume_flow_rate",
        # y="arithmetic_mean_of_difference_of_static_pressure_between_fan_outlet_and_fan_inlet",
        y="arithmetic_mean_of_difference_of_total_pressure_between_fan_outlet_and_fan_inlet",
        xlabel=None,
        ylabel=None,
        label="$p_{tot} (n=600 rpm)$",
        marker=".",
        linestyle='-',
        ax=dax.ax,
        verbose=True
    )
    fan_curve.errorbar(
        x="arithmetic_mean_of_fan_volume_flow_rate",
        y="arithmetic_mean_of_difference_of_static_pressure_between_fan_outlet_and_fan_inlet",
        xlabel=None,
        ylabel=None,
        label="$p_{stat} (n=600 rpm)$",
        marker=".",
        linestyle='-',
        ax=dax.ax,
    )
    fan_curve_1000.errorbar(
        x="arithmetic_mean_of_fan_volume_flow_rate",
        # y="arithmetic_mean_of_difference_of_static_pressure_between_fan_outlet_and_fan_inlet",
        y="arithmetic_mean_of_difference_of_total_pressure_between_fan_outlet_and_fan_inlet",
        xlabel=None,
        ylabel=None,
        label="$p_{tot} (n=1000 rpm)$",
        marker=".",
        linestyle='-',
        ax=dax.ax,
        verbose=True
    )
    fan_curve_1000.errorbar(
        x="arithmetic_mean_of_fan_volume_flow_rate",
        y="arithmetic_mean_of_difference_of_static_pressure_between_fan_outlet_and_fan_inlet",
        xlabel=None,
        ylabel="Pressure difference / Pa",
        label="$p_{stat} (n=1000 rpm)$",
        marker=".",
        linestyle='-',
        ax=dax.ax,
    )
    plt.legend()
    plt.tight_layout()

## Serialization of fan curves

Since we model the operation points and fan curve semantically, we can serialize it as TTL in order to share it with others or machines.

In [ ]:
ttl = fan_curve.serialize()

In [ ]:
print(ttl)